TODO
* Complete function documentation.
* Jail strategy is hard-coded as, in order, use a Get-Out-Of-Jail card, pay the fine, and roll the dice. In the future, strategy selection should be able to vary across players and be context-sensitive.
* Default method is simplistic. Need to code logic for what happens to defaulter's property.

### Import modules

In [10]:
import csv
import pandas as pd
import numpy as np
#import parameters as p

### Define functions

In [4]:
def get_players(n_players):
        
    # Ensure number of players is acceptable
    assert n_players < 2, 'A game must have at least 2 players.'
    assert n_players > 8, 'A game must have no more than 8 players.'
    
    class Player(object):
        
        def __init__(self, player_id):
            self.id = player_id  # Identification number
            self.cash = 1500     # Cash on hand
            self.properties = [] # List of properties
            self.position = 0    # Board position
            self.jail_cards = 0  # Number of "Get Out Of Jail Free" cards
            self.jail_turns = 0  # Number of remaining turns in jail
            self.jail_strtg = '' # Jail strategy
        
        def move(self, roll, verbose=False):
            self.position += roll
            if self.position >= 40:
                self.position -= 40
                self.cash += 200
            if verbose:
                print 'Player {} to space: {}'.format(self.id, self.position)
        
        def buy(self, prop_id, price):
            self.properties.append(prop_id)
            self.cash -= price
        
        def pay(self, payee, payment):
            self.cash -= payment
            players[payee].cash += payment
        
        def go_to_jail(self):
            self.position = 10
            self.jail_turns = 3
        
        def choose_jail_strtg(self, rolled_double):
            # Use get
            if self.jail_cards > 0:
                self.jail_strtg = 'card'
                self.jail_turns = 0
                self.jail_cards -= 1
            elif self.cash >= 50:
                self.jail_strtg = 'pay'
                self.jail_turns = 0
                self.cash -= 50
            else:
                self.jail_strtg = 'roll'
                if rolled_double:
                    self.jail_turns = 0
                else:
                    self.jail_turns -= 1
                    if self.jail_turns == 0:
                        self.cash -= 50
                        
        def default(self):
            del players[self.id]
            
    # Create list of players
    return [Player(p) for p in range(n_players)]

In [54]:
def get_board(board_file):
        
    class Property(object):
        def __init__(self, name, position, price, rent):
            self.name = name
            self.position = position
            self.price = price
            self.price_mortgage = price / 2
            self.rent = rent
            self.rent_now = rent
            self.mortgage = False
            self.owner = 0

    class Street(Property):
        def __init__(self, name, position, color, price,
                     price_building, rent, rent_building):
                         Property.__init__(self, name, position, price, rent)
                         self.color = color
                         self.price_building = price_building
                         self.rent_monopoly = rent * 2
                         self.rent_building = rent_building
                         self.n_building = 0

    class Railroad(Property):
        def __init__(self, name, position, price, rent):
            Property.__init__(self, name, position, price, rent)
            self.rent_double = rent * 2
            self.rent_triple = self.rent_double * 2
            self.rent_monopoly = self.rent_triple * 2
    
    class Utility(Property):
        def __init__(self, name, position, price, rent):
            Property.__init__(self, name, position, price, rent)
            self.rent_monopoly = rent + 6
    
    class Tax(object):
        def __init__(self, price):
            self.price = price

    class Card(object):
        pass  
    
    class Chance(object):
        pass
    
    class Chest(object):
        pass
    
    class Jail(object):
        pass
    
    class Idle(object):
        pass
    
    board = []
    for _, r in pd.read_csv(board_file).iterrows():
        if r['class'] == 'Street':
            board.append(Street(r['name'], r['position'], r['color'], r['price_buy'], r['price_build'], r['rent'],
                                [row[j] for j in ['rent_build_{}'.format(i + 1) for i in range(5)]]))
        elif r['class'] == 'Railroad':
            board.append(Railroad(r['name'], r['position'], r['price_buy'], r['rent']))
        elif r['class'] == 'Utility':
            board.append(Utility(r['name'], r['position'], r['price_buy'], r['rent']))
        elif r['class'] == 'Tax':
            board.append(Tax(r['price_buy']))
        elif r['class'] == 'Chance':
            board.append(Chance())
        elif r['class'] == 'Chest':
            board.append(Chest())
        elif r['class'] == 'Jail':
            board.append(Jail())
        elif r['class'] == 'Idle':
            board.append(Idle())
    
    return board